In [ ]:
"""
Created on Thu Sep 08 11:15 2022

This script is to automate a bit the cross-validation and avoiding having to do it in a notebook

Author: Clara Burgard
"""

import numpy as np
import xarray as xr
import pandas as pd
from tqdm.notebook import trange, tqdm
import glob
import datetime
import time
import sys

import tensorflow as tf
from tensorflow import keras
import basal_melt_neural_networks.model_functions as modf
import basal_melt_neural_networks.prep_input_data as indat

In [ ]:
######### READ IN OPTIONS

mod_size = 'medium' #'mini', 'small', 'medium', 'large', 'extra_large'
tblock_out = 1
isf_out =0
TS_opt = 'extrap' # extrap, whole, thermocline
norm_method = 'std' # std, interquart, minmax
exp_name = 'testest'

# INPUT
# onlyTSdraft: 'corrected_isfdraft','theta_in','salinity_in'
# TSdraftbotandiceddandwcd: 'corrected_isfdraft','water_col_depth','theta_in','salinity_in','theta_bot','salinity_bot'

In [ ]:
######### READ IN DATA

inputpath_data = '/bettik/burgardc/DATA/NN_PARAM/interim/INPUT_DATA/'
outputpath_nn_models = '/bettik/burgardc/DATA/NN_PARAM/interim/NN_MODELS/experiments/'
outputpath_doc = '/bettik/burgardc/SCRIPTS/basal_melt_neural_networks/custom_doc/experiments/'

tblock_dim = range(1,14)
isf_dim = [10,11,12,13,18,22,23,24,25,30,31,33,38,39,40,42,43,44,45,47,48,51,52,53,54,55,58,61,65,66,69,70,71,73,75]

if (tblock_out > 0) and (isf_out == 0):
    path_model = outputpath_nn_models+'CV_TBLOCK/'
    
elif (isf_out > 0) and (tblock_out == 0):
    path_model = outputpath_nn_models+'CV_ISF/'
    
else:
    print("I do not know what to do with both tblock and isf left out! ")

#new_path_doc = outputpath_doc+timetag+'/'
#if not os.path.isdir(new_path_doc):
#    print("I did not find this folder ("+timetag+") in doc folder! :( ")

if TS_opt == 'extrap':
    inputpath_CVinput = inputpath_data+'EXTRAPOLATED_ISFDRAFT_CHUNKS_CV/'
elif TS_opt == 'whole':
    inputpath_CVinput = inputpath_data+'WHOLE_PROF_CHUNKS_CV/'
elif TS_opt == 'thermocline':
    inputpath_CVinput = inputpath_data+'THERMOCLINE_CHUNKS_CV/'


if TS_opt == 'extrap':
    
    data_train_orig_norm = xr.open_dataset(inputpath_CVinput + 'train_data_CV_noisf'+str(isf_out).zfill(3)+'_notblock'+str(tblock_out).zfill(3)+'.nc')
    data_val_orig_norm = xr.open_dataset(inputpath_CVinput + 'val_data_CV_noisf'+str(isf_out).zfill(3)+'_notblock'+str(tblock_out).zfill(3)+'.nc') 
    data_train_addvar1_norm = xr.open_dataset(inputpath_CVinput + 'train_addvar1_CV_noisf'+str(isf_out).zfill(3)+'_notblock'+str(tblock_out).zfill(3)+'.nc')
    data_val_addvar1_norm = xr.open_dataset(inputpath_CVinput + 'val_addvar1_CV_noisf'+str(isf_out).zfill(3)+'_notblock'+str(tblock_out).zfill(3)+'.nc')
    

    print('here1')
    data_train_norm = xr.merge([data_train_orig_norm[['corrected_isfdraft','theta_in','salinity_in','melt_m_ice_per_y']], 
                                data_train_addvar1_norm[['water_col_depth','theta_bot','salinity_bot']]])
    print('here2')
    data_val_norm = xr.merge([data_val_orig_norm[['corrected_isfdraft','theta_in','salinity_in','melt_m_ice_per_y']], 
                                data_val_addvar1_norm[['water_col_depth','theta_bot','salinity_bot']]])
    
    ## prepare input and target
            
    print('here3')
    y_train_norm = data_train_norm['melt_m_ice_per_y'].sel(norm_method=norm_method).load()
    x_train_norm = data_train_norm.drop_vars(['melt_m_ice_per_y']).sel(norm_method=norm_method).to_array().load()
    #print('here4')
    #x_train_norm = data_train_norm[['corrected_isfdraft','water_col_depth','theta_in','salinity_in','theta_bot','salinity_bot']].sel(norm_method=norm_method).to_array().load()
    
    print('here5')
    y_val_norm = data_val_norm['melt_m_ice_per_y'].sel(norm_method=norm_method).load()
    x_val_norm = data_val_norm.drop_vars(['melt_m_ice_per_y']).sel(norm_method=norm_method).to_array().load()
    #print('here6')
    #x_val_norm = data_val_norm[['corrected_isfdraft','water_col_depth','theta_in','salinity_in','theta_bot','salinity_bot']].sel(norm_method=norm_method).to_array().load()

In [ ]:
data_train_orig_norm[['corrected_isfdraft','theta_in','salinity_in','melt_m_ice_per_y']]

In [ ]:
data_train_addvar1_norm[['water_col_depth','theta_bot','salinity_bot']]

In [ ]:
data_val_orig_norm[['corrected_isfdraft','theta_in','salinity_in','melt_m_ice_per_y']]